In [1]:
from pinecone import Pinecone, ServerlessSpec
from pinecone_text.sparse import SpladeEncoder
from langchain_community.document_loaders import JSONLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_fireworks import ChatFireworks
from langchain_pinecone import PineconeVectorStore
import torch
from sentence_transformers import SentenceTransformer
import pandas as pd
from tqdm import tqdm
import os
from dotenv import load_dotenv

load_dotenv()

/home/lavender/02_SaleChatbot/venv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [2]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
FIREWORKS_API_KEY = os.getenv("FIREWORKS_API_KEY")
INDEX_NAME = "salechatbot"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
pc = Pinecone(api_key=PINECONE_API_KEY)

In [4]:
index = pc.Index(INDEX_NAME)

In [5]:
splade = SpladeEncoder()

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


In [6]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/clip-ViT-B-32")

In [7]:
from langchain_community.retrievers import (
    PineconeHybridSearchRetriever,
)

retriever = PineconeHybridSearchRetriever(
    embeddings=embeddings, sparse_encoder=splade, index=index, top_k=10
)

In [8]:
template = """You are a sale assistant. 
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""

custom_rag_prompt = PromptTemplate.from_template(template)

In [15]:
llm = ChatFireworks(model="accounts/fireworks/models/llama-v3p1-8b-instruct", max_retries=3, temperature=0.0)

In [16]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [23]:
for chunk in rag_chain.stream("Tôi muốn mua bàn phím gaming."):
    print(chunk, end="", flush=True)

Bạn có thể xem xét bàn phím gaming Tech77 với giá 620.000 đồng hoặc bàn phím game Fuhlen L411 với giá 200.000 đồng. Cả hai đều có sẵn. Thanks for asking!

In [24]:
examples = retriever.invoke("Tôi muốn mua bàn phím gaming.")

In [26]:
_=[print(example.metadata["name"]) for example in examples]

Bàn Phím Cơ Gaming dây usb GK102 Hotswap chống ồn cho máy tính laptop hàng nhập khẩu
Bàn Phím Game Fuhlen L411 - Hàng Chính Hãng
Bàn phím giả cơ K518 kèm chuột, bàn phím máy tính chống nước led rgb chơi game làm việc văn phòng cho PC laptop - hàng chính hãng
Bộ Bàn Phím Chuột Không Dây Besti BTY01 Nhỏ Gọn, Tiện Lợi Mang Theo- Hàng Chính Hãng
Miếng dán bàn phím tiếng Hàn Quốc Hàng Nhập Khẩu
Bao da Book cover bàn phím dành cho samsung Tab S7+/TAB S7 FE/TAB S8+ SLIM EF-DT730 Hàng chính hãng
Cuộn mút xốp nhiều màu bọc góc bàn chữ U bo góc bảo vệ em bé tránh va chạm trầy xước Legaxi
Bàn phím cơ không dây Newmen GM610 - Hàng chính hãng
Bàn phím Magic Keyboard Apple MK2A3 (US keyboard)
Đầu Android TVbox Mytv net Phiên bản Ram 2G/16G 4G/32G điều khiển IR- Xem 200 Kênh truyền hình miễn phí - Hàng Chính Hãng


In [27]:
# while True:
#     user_input = input("User")
#     print("User:", user_input)
#     if user_input in ["q", "quit", "bye"]:
#         break

#     print("Chatbot:", end=" ", flush=True)
#     for chunk in rag_chain.stream(user_input):
#         print(chunk, end="", flush=True)
#     print()